In [1]:
from chase.base import *
from chase.utils import *
from chase.fit import *
from mypy.explib.frey2014 import frey2014
pd.set_option('display.max_colwidth', 100)

In [ ]:
arr = []
for sid in frey2014.subjects():
    print sid
    data = frey2014.subject_fitdata(sid)
    for d in data:
        arr.append([sid, d['gid'], d['group'], d['data'][0,0], d['data'][0,1]])
        
    df = pd.DataFrame(arr, columns=['subject', 'problem', 'group', 'choice', 'samplesize'])
df.to_csv('frey_data.csv')

In [2]:
df = frey2014.load_data()

In [35]:
df.to_csv('/Users/markant/Dropbox/SequentialSamplingDFE/frey_trial_data.csv')

In [2]:
data = pd.read_csv('data/frey_data.csv', index_col=0)
problems = {gid: frey2014.get_options(gid) for gid in data.problem.unique()}
data.head(10)

,subject,problem,group,choice,samplesize
0,1,S1G1,0,0,12
1,1,S2G1,0,1,10
2,1,S3G1,0,0,10
3,1,S4G1,0,0,10
4,1,S5G1,0,0,10
5,1,S6G1,0,1,10
6,1,S7G1,0,0,10
7,1,S8G1,0,0,12
8,1,S9G1,0,0,10
9,1,S10G1,0,0,10


# Fit individual subjects

In [4]:
data.subject.unique()[10]

13

In [7]:
# a list of possible free parameters and their ranges
# and starting point (optional)
PARS = {'theta': [1, 10],
        'p_stay': [0, 1],
        'tau': [0, 1, .5],
        'prelec_gamma': [0, 5, 1],
        'prelec_elevation': [0, 5, 1],
        'pow_gain': [0., np.inf, 1.],
        'w_loss': [0., np.inf, 1.]}

# parameters that are set to fixed values 
FIXED = {'c': 0.5}

N_ITER = 3
OUTDIR = 'chase_fitresults_frey_individual'

# a list of different parameter combinations that will
# be fit
PARSETS = [['theta', 'p_stay', 'tau'],
           ['theta', 'p_stay', 'tau', 'prelec_gamma', 'prelec_elevation'],
           ['theta', 'p_stay', 'tau', 'pow_gain', 'w_loss'],
           ]

In [20]:
sset = [2, 6, 9, 12, 13]

In [8]:
for sid in data.subject.unique()[:5]:

    SIM_ID = 'frey_individual_subj=%s' % sid

    for parset in PARSETS:

        fitting = {p: PARS[p] for p in parset}

        # initialize the model
        m = CHASEModel(drift='cpt',
                       startdist='laplace')

        # fit
        results = fit_mlh(m, problems, data[data.subject==sid], 
                          SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR)

        print results.sort('nllh').head(1)

frey_individual_subj=1(p_stay,tau,theta|c=0.5)
0/30
theta: 1.0
iteration            0
success           True
nllh          334.6236
k                    3
N                   84
bic           682.5397
theta                1
p_stay       0.9030023
tau          0.1631628
Name: 0, dtype: object
1/30
theta: 2.0
iteration               0
success              True
nllh              319.718
k                       3
N                      84
bic              652.7284
theta                   2
p_stay          0.6658739
tau          2.354073e-07
Name: 1, dtype: object
2/30
theta: 3.0
iteration               0
success              True
nllh             318.0681
k                       3
N                      84
bic              649.4286
theta                   3
p_stay          0.2829321
tau          7.979239e-08
Name: 2, dtype: object
3/30
theta: 4.0
iteration               0
success              True
nllh             310.6448
k                       3
N                      84
bic            

In [22]:
allresults = pd.DataFrame(columns=['sim_id', 'k', 'N', 'nllh', 'bic'])
for parset in PARSETS:
    fitting = {p: PARS[p] for p in parset}
    allresults.loc[allresults.shape[0]] = best_result('frey_individual_subj=%s' % sid, FIXED, fitting, outdir=OUTDIR, nopars=True)

allresults.sort('bic')

,sim_id,k,N,nllh,bic
0,"frey_individual_subj=15(p_stay,tau,theta|delta=0.5)",3,84,328.382953,670.058356
2,"frey_individual_subj=15(p_stay,pow_gain,tau,theta|delta=0.5)",4,84,326.263444,670.250156
1,"frey_individual_subj=15(p_stay,prelec_gamma,tau,theta|delta=0.5)",4,84,327.635621,672.994509
